In [ ]:
# Step 4: Hybrid Explanation Generator - USING SHAP
# Key changes from original:
# 1. Replace DeepLIFT with SHAP KernelExplainer
# 2. Use background data for SHAP baseline
# 3. Extract class 1 (Important) attributions

import os
import warnings
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from pathlib import Path
import joblib
from typing import Dict, List, Tuple, Optional
import json
import pickle

warnings.filterwarnings('ignore')

print("="*70)
print("STEP 4: HYBRID EXPLANATION GENERATOR (USING SHAP)")
print("="*70)

# ==================== CONFIGURATION ====================

LSTM_MODEL_PATH = '../step1_lstm_xai/best_lstm.pt'
SCALER_PATH = '../step1_lstm_xai/scaler.joblib'
CAUSAL_GRAPH_PATH = '../step2_causal_discovery/causal_graph.gpickle'
DATA_PATH = '../UNSW_NB15_training-set.csv'

FEATURE_NAMES = [
    'dur', 'proto', 'service', 'state', 'spkts', 'dpkts',
    'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload',
    'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit',
    'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
    'smean', 'dmean', 'trans_depth', 'response_body_len',
    'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm',
    'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login',
    'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst',
    'is_sm_ips_ports'
]

CAUSAL_FEATURES = [
    'proto', 'sttl', 'state', 'dtcpb', 'is_sm_ips_ports',
    'dttl', 'stcpb', 'service', 'dwin', 'swin'
]

MISSING_VALUE_INDICATOR = -1.0

# ==================== LOAD MODELS AND DATA ====================
print("\nLoading models and data...")

# Load LSTM model architecture
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

# Load trained model
device = torch.device('cpu')
input_size = len(FEATURE_NAMES)
model = LSTMClassifier(input_size=input_size).to(device)

if Path(LSTM_MODEL_PATH).exists():
    model.load_state_dict(torch.load(LSTM_MODEL_PATH, map_location=device))
    model.eval()
    print(f"✓ Loaded LSTM model from {LSTM_MODEL_PATH}")
else:
    print(f"⚠ Warning: {LSTM_MODEL_PATH} not found. Using untrained model.")

# Load scaler
if Path(SCALER_PATH).exists():
    scaler = joblib.load(SCALER_PATH)
    print(f"✓ Loaded scaler from {SCALER_PATH}")
else:
    print(f"⚠ Warning: {SCALER_PATH} not found.")
    scaler = None

# Load causal graph
if Path(CAUSAL_GRAPH_PATH).exists():
    causal_graph = pickle.load(open(CAUSAL_GRAPH_PATH, 'rb'))
    print(f"✓ Loaded causal graph from {CAUSAL_GRAPH_PATH}")
else:
    print(f"⚠ Warning: {CAUSAL_GRAPH_PATH} not found.")
    causal_graph = nx.DiGraph()

# Load data
if Path(DATA_PATH).exists():
    df = pd.read_csv(DATA_PATH)
    print(f"✓ Loaded data from {DATA_PATH}: {df.shape}")
    
    # Encode categorical features
    from sklearn.preprocessing import LabelEncoder
    drop_cols = ['id', 'attack_cat']
    for col in drop_cols:
        if col in df.columns:
            df = df.drop(columns=[col])
    
    if 'label' in df.columns and 'Label' not in df.columns:
        df['Label'] = df['label']
    
    categorical_cols = ['proto', 'service', 'state']
    for col in categorical_cols:
        if col in df.columns and df[col].dtype == 'object':
            le = LabelEncoder()
            df[col] = df[col].fillna('unknown')
            df[col] = le.fit_transform(df[col].astype(str))
    
    print("✓ Categorical features encoded")
else:
    print(f"⚠ Warning: {DATA_PATH} not found.")
    df = None

# ==================== SHAP EXPLAINER SETUP ====================
print("\n" + "="*70)
print("XAI COMPONENT: SHAP Explainer")
print("="*70)

# Initialize SHAP explainer
shap_explainer = None
background_data = None

if df is not None and scaler is not None:
    try:
        import shap
        
        # Get background data (sample from training data)
        print("\nInitializing SHAP explainer...")
        available_features = [f for f in FEATURE_NAMES if f in df.columns]
        
        if len(available_features) == len(FEATURE_NAMES):
            # Sample 100 background samples
            background_indices = np.random.choice(len(df), size=min(100, len(df)), replace=False)
            background_data_raw = df.iloc[background_indices][FEATURE_NAMES].values
            
            # Scale background data
            background_data = scaler.transform(background_data_raw)
            print(f"  Background data shape: {background_data.shape}")
            
            # Define model prediction function for SHAP
            def model_predict_shap(x):
                """
                Prediction function for SHAP
                Input: numpy array (n_samples, n_features) - SCALED
                Output: numpy array (n_samples, n_classes) - probabilities
                """
                if x.ndim == 2:
                    # Add sequence dimension: (batch, features) -> (batch, 1, features)
                    x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(1).to(device)
                else:
                    x_tensor = torch.tensor(x, dtype=torch.float32).to(device)
                
                with torch.no_grad():
                    output = model(x_tensor)
                    probs = torch.softmax(output, dim=1).cpu().numpy()
                
                return probs
            
            # Initialize KernelExplainer
            shap_explainer = shap.KernelExplainer(
                model_predict_shap,
                background_data
            )
            print("✓ SHAP KernelExplainer initialized")
            
        else:
            print(f"⚠ Warning: Missing features. SHAP unavailable.")
            
    except ImportError:
        print("⚠ Warning: SHAP not installed. Install with: pip install shap")
    except Exception as e:
        print(f"⚠ Warning: SHAP initialization failed: {e}")

# ==================== XAI COMPONENT WITH SHAP ====================

def compute_shap_attribution(shap_explainer, alert_tensor_scaled, n_samples=100):
    """
    Compute SHAP attributions for a single alert
    
    Args:
        shap_explainer: SHAP KernelExplainer instance
        alert_tensor_scaled: numpy array (n_features,) - MUST BE SCALED
        n_samples: Number of samples for SHAP approximation
    
    Returns:
        numpy array of feature attributions for class 1 (Important)
    """
    if shap_explainer is None:
        print("    ⚠ SHAP explainer not available, using gradient fallback")
        return compute_gradient_attribution_fallback(alert_tensor_scaled)
    
    try:
        # SHAP expects 2D input: (1, n_features)
        alert_2d = alert_tensor_scaled.reshape(1, -1)
        
        # Compute SHAP values
        shap_values = shap_explainer.shap_values(alert_2d, nsamples=n_samples)
        
        # Debug: Check shape
        print(f"    [SHAP DEBUG] Raw SHAP output type: {type(shap_values)}")
        if isinstance(shap_values, list):
            print(f"    [SHAP DEBUG] List length: {len(shap_values)}")
            for i, sv in enumerate(shap_values):
                print(f"    [SHAP DEBUG] Class {i} shape: {sv.shape if hasattr(sv, 'shape') else type(sv)}")
        else:
            print(f"    [SHAP DEBUG] Single output shape: {shap_values.shape}")
        
        # Extract class 1 (Important) attributions
        if isinstance(shap_values, list):
            # Multi-class output: [class_0_shap, class_1_shap]
            attributions = shap_values[1]  # Get class 1
            
            # Handle different dimensionalities
            if attributions.ndim > 1:
                attributions = attributions.flatten()  # Flatten to 1D
        else:
            # Single output array from KernelExplainer
            # Shape can be: (1, n_features) or (1, n_features, n_classes)
            
            if shap_values.ndim == 3:
                # Shape: (n_samples, n_features, n_classes)
                # Extract class 1 (Important) for first sample
                print(f"    [SHAP DEBUG] Extracting class 1 from shape {shap_values.shape}")
                attributions = shap_values[0, :, 1]  # [sample=0, all_features, class=1]
            elif shap_values.ndim == 2:
                # Shape: (n_samples, n_features)
                # Already correct, just get first sample
                attributions = shap_values[0, :]
            else:
                # Shape: (n_features,)
                attributions = shap_values
        
        print(f"    [SHAP DEBUG] Final attribution shape: {attributions.shape}")
        
        # Final validation
        expected_features = len(alert_tensor_scaled)
        if len(attributions) != expected_features:
            print(f"    ⚠ WARNING: Attribution length {len(attributions)} != expected {expected_features}")
            print(f"    Truncating/padding to match feature count")
            if len(attributions) > expected_features:
                attributions = attributions[:expected_features]
            else:
                # Pad with zeros
                attributions = np.pad(attributions, (0, expected_features - len(attributions)))
        
        return attributions
        
    except Exception as e:
        print(f"    ⚠ SHAP failed: {e}. Using gradient fallback.")
        import traceback
        traceback.print_exc()
        return compute_gradient_attribution_fallback(alert_tensor_scaled)

def compute_gradient_attribution_fallback(alert_features_scaled):
    """
    Fallback: Simple gradient-based attribution
    """
    alert_tensor = torch.tensor(alert_features_scaled, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    alert_tensor.requires_grad = True
    
    output = model(alert_tensor)
    pred_class = output.argmax(dim=1).item()
    
    model.zero_grad()
    output[0, pred_class].backward()
    
    gradients = alert_tensor.grad.squeeze().detach().cpu().numpy()
    values = alert_tensor.squeeze().detach().cpu().numpy()
    
    attributions = gradients * values
    return attributions

def is_missing_value(value, threshold=MISSING_VALUE_INDICATOR):
    """Check if a feature value represents missing/NA data"""
    return abs(value - threshold) < 1e-6

def generate_xai_explanation(model, alert_features, feature_names, top_k=5, scaler=None, shap_explainer=None):
    """
    Generate XAI explanation using SHAP
    
    Args:
        model: LSTM model
        alert_features: numpy array of feature values (UNSCALED)
        feature_names: list of feature names
        top_k: number of top features
        scaler: MinMaxScaler
        shap_explainer: SHAP KernelExplainer instance
    
    Returns:
        Dictionary with XAI results
    """
    # Scale features
    if scaler is not None:
        alert_features_scaled = scaler.transform(alert_features.reshape(1, -1)).flatten()
    else:
        alert_features_scaled = alert_features
    
    # Get prediction
    alert_tensor = torch.tensor(alert_features_scaled, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(alert_tensor)
        probs = torch.softmax(output, dim=1)[0]
        pred_class = output.argmax(dim=1).item()
        confidence = probs[pred_class].item()
    
    print(f"  [DEBUG] Prediction: {pred_class}, Confidence: {confidence:.4f}")
    
    # Compute SHAP attributions
    attributions = compute_shap_attribution(shap_explainer, alert_features_scaled, n_samples=50)
    
    print(f"  [DEBUG] Attribution range: [{attributions.min():.4f}, {attributions.max():.4f}]")
    
    # Combine features with attributions
    feature_importance = []
    for name, attr, value in zip(feature_names, attributions, alert_features):
        feature_importance.append({
            'feature': name,
            'importance': float(attr),
            'value': float(value),
            'abs_importance': float(abs(attr)),
            'is_missing': is_missing_value(value)
        })
    
    # Sort by absolute importance
    feature_importance.sort(key=lambda x: x['abs_importance'], reverse=True)
    
    # Filter out missing values
    feature_importance_present = [f for f in feature_importance if not f['is_missing']]
    
    missing_count = len([f for f in feature_importance if f['is_missing']])
    present_count = len(feature_importance_present)
    
    print(f"  [DEBUG] Features: {present_count} present, {missing_count} missing")
    
    top_features = feature_importance_present[:top_k] if len(feature_importance_present) >= top_k else feature_importance[:top_k]
    
    return {
        'prediction': 'Important' if pred_class == 1 else 'Irrelevant',
        'confidence': confidence,
        'pred_class': pred_class,
        'top_features': top_features,
        'all_features': feature_importance,
        'num_missing_features': missing_count,
        'num_present_features': present_count
    }

# ==================== CAUSAL COMPONENT ====================
# [Keep all causal analysis functions from original code]
print("\n" + "="*70)
print("CAUSAL COMPONENT: Root Cause Analysis")
print("="*70)

def find_root_causes(graph, target_feature):
    """Find all root causes of target"""
    if target_feature not in graph:
        return []
    ancestors = nx.ancestors(graph, target_feature)
    return [node for node in ancestors if graph.in_degree(node) == 0]

def find_causal_path(graph, source, target):
    """Find shortest causal path"""
    if source not in graph or target not in graph:
        return None
    try:
        return nx.shortest_path(graph, source, target)
    except nx.NetworkXNoPath:
        return None

def get_direct_causes(graph, feature):
    """Get direct causes (parents)"""
    if feature not in graph:
        return []
    return list(graph.predecessors(feature))

def analyze_causal_chain(graph, target_feature, alert_data, all_feature_names):
    """Analyze causal chains leading to target"""
    if target_feature not in graph:
        return {
            'target': target_feature,
            'in_graph': False,
            'root_causes': [],
            'causal_paths': [],
            'direct_causes': [],
            'reason': 'Feature not in causal graph'
        }
    
    root_causes = find_root_causes(graph, target_feature)
    
    causal_paths = []
    for root in root_causes:
        path = find_causal_path(graph, root, target_feature)
        if path:
            path_with_values = [
                {'feature': f, 'value': alert_data.get(f, 'N/A')}
                for f in path
            ]
            causal_paths.append({
                'root': root,
                'path': path,
                'path_with_values': path_with_values,
                'length': len(path)
            })
    
    direct_causes = get_direct_causes(graph, target_feature)
    direct_causes_with_values = [
        {'feature': c, 'value': alert_data.get(c, 'N/A')}
        for c in direct_causes
    ]
    
    return {
        'target': target_feature,
        'in_graph': True,
        'root_causes': root_causes,
        'causal_paths': causal_paths,
        'direct_causes': direct_causes_with_values,
        'num_paths': len(causal_paths)
    }

# ==================== HYBRID EXPLAINER ====================
print("\n" + "="*70)
print("HYBRID EXPLAINER: Combining SHAP + Causal")
print("="*70)

class HybridExplainer:
    """Combines SHAP and Causal Analysis"""
    
    def __init__(self, model, causal_graph, feature_names, scaler=None, shap_explainer=None):
        self.model = model
        self.graph = causal_graph
        self.feature_names = feature_names
        self.scaler = scaler
        self.shap_explainer = shap_explainer
    
    def explain(self, alert_data, alert_id=None):
        """Generate hybrid explanation"""
        if isinstance(alert_data, dict):
            alert_features = np.array([alert_data.get(f, 0) for f in self.feature_names])
            alert_dict = alert_data
        else:
            alert_features = alert_data
            alert_dict = {f: v for f, v in zip(self.feature_names, alert_features)}
        
        # Get XAI explanation using SHAP
        xai_results = generate_xai_explanation(
            self.model,
            alert_features,
            self.feature_names,
            top_k=5,
            scaler=self.scaler,
            shap_explainer=self.shap_explainer
        )
        
        # Analyze causal chains
        causal_analyses = []
        for feat_info in xai_results['top_features']:
            feature_name = feat_info['feature']
            if feature_name in self.graph:
                causal_analysis = analyze_causal_chain(
                    self.graph, feature_name, alert_dict, self.feature_names
                )
            else:
                causal_analysis = {
                    'target': feature_name,
                    'in_graph': False,
                    'root_causes': [],
                    'causal_paths': [],
                    'direct_causes': [],
                    'reason': 'Feature not in SOC analyst-determined causal graph'
                }
            causal_analyses.append(causal_analysis)
        
        # Label causal analysis
        label_causal = None
        if 'Label' in self.graph:
            label_causal = analyze_causal_chain(
                self.graph, 'Label', alert_dict, self.feature_names
            )
        
        # Generate recommendations
        recommendations = self._generate_recommendations(
            xai_results, causal_analyses, alert_dict
        )
        
        # Return HybridExplanation object
        explanation = HybridExplanation(
            alert_id=alert_id,
            alert_data=alert_dict,
            xai_results=xai_results,
            causal_analyses=causal_analyses,
            label_causal=label_causal,
            recommendations=recommendations
        )
        
        return explanation
    
    def _generate_recommendations(self, xai_results, causal_analyses, alert_data):
        """
        Generate actionable recommendations based on XAI and causal analysis
        """
        recommendations = {
            'severity': 'MEDIUM',
            'immediate_actions': [],
            'investigation_steps': [],
            'root_cause_mitigation': [],
            'severity_reasoning': ''
        }
        
        confidence = xai_results.get('confidence', 0.5)
        prediction = xai_results.get('prediction', 'Unknown')
        top_features = xai_results.get('top_features', [])
        
        if not top_features:
            recommendations['immediate_actions'].append(
                "Alert requires manual review - feature analysis unavailable"
            )
            return recommendations
        
        # Severity assessment for imbalanced dataset
        if prediction == 'Important':
            if confidence > 0.95:
                recommendations['severity'] = 'HIGH'
                recommendations['severity_reasoning'] = (
                    f"High-confidence Important alert ({confidence:.1%}). "
                    "Given dataset imbalance (1.5% Important), high confidence indicates strong evidence."
                )
            elif confidence > 0.80:
                recommendations['severity'] = 'MEDIUM'
                recommendations['severity_reasoning'] = (
                    f"Medium-confidence Important alert ({confidence:.1%}). "
                    "Requires further investigation to validate."
                )
            else:
                recommendations['severity'] = 'LOW'
                recommendations['severity_reasoning'] = (
                    f"Low-confidence Important alert ({confidence:.1%}). "
                    "Model uncertain - likely borderline case requiring manual review."
                )
        else:
            if confidence > 0.95:
                recommendations['severity'] = 'MINIMAL'
                recommendations['severity_reasoning'] = (
                    f"High-confidence Normal alert ({confidence:.1%}). "
                    "Can likely be safely dismissed."
                )
            elif confidence > 0.80:
                recommendations['severity'] = 'LOW'
                recommendations['severity_reasoning'] = (
                    f"Medium-confidence Normal alert ({confidence:.1%}). "
                    "Quick review recommended to confirm."
                )
            else:
                recommendations['severity'] = 'MEDIUM'
                recommendations['severity_reasoning'] = (
                    f"Low-confidence Normal alert ({confidence:.1%}). "
                    "Uncertain classification - may be Important. Investigate carefully."
                )
        
        # Analyze top features for specific actions
        for feat in top_features:
            feature = feat.get('feature', '')
            value = feat.get('value', 0)
            importance = feat.get('importance', 0)
            
            if feat.get('is_missing', False):
                continue
            
            # STTL
            if feature == 'sttl':
                if value < 30:
                    recommendations['immediate_actions'].append(
                        f"⚠ CRITICAL: Abnormal TTL ({int(value)}) - Strong attack indicator"
                    )
                    recommendations['investigation_steps'].append(
                        "Low TTL suggests packet spoofing or routing manipulation"
                    )
            
            # STATE
            if feature == 'state':
                state_map = {0: 'CON', 1: 'ECO', 2: 'FIN', 3: 'INT', 
                            4: 'PAR', 5: 'REQ', 6: 'RST', 7: 'URN', 8: 'no'}
                state_name = state_map.get(int(value), 'Unknown')
                
                if state_name in ['REQ', 'INT']:
                    recommendations['immediate_actions'].append(
                        f"⚠ Suspicious connection state ({state_name})"
                    )
            
            # PROTO
            if feature == 'proto':
                proto_map = {6: 'TCP', 17: 'UDP', 1: 'ICMP'}
                proto_name = proto_map.get(int(value), f'Protocol {int(value)}')
                
                if value == 1:
                    recommendations['immediate_actions'].append(
                        "⚠ ICMP protocol - Often used in reconnaissance or DDoS"
                    )
            
            # SCANNING INDICATOR
            if feature == 'is_sm_ips_ports' and value == 1:
                recommendations['immediate_actions'].append(
                    "⚠ PORT SCAN DETECTED - Same source/dest IPs and ports"
                )
        
        # Causal-based recommendations
        for causal in causal_analyses:
            if causal.get('in_graph', False) and causal.get('root_causes'):
                target = causal['target']
                roots = causal['root_causes']
                
                recommendations['root_cause_mitigation'].append(
                    f"Root causes of {target}: {', '.join(roots)}"
                )
        
        # Default actions
        if not recommendations['immediate_actions'] and prediction == 'Important':
            recommendations['immediate_actions'].append(
                "Review alert details and correlate with other security events"
            )
        
        return recommendations

class HybridExplanation:
    """Container for hybrid explanation results"""
    
    def __init__(self, alert_id, alert_data, xai_results, causal_analyses, 
                 label_causal, recommendations):
        self.alert_id = alert_id
        self.alert_data = alert_data
        self.xai = xai_results
        self.causal = causal_analyses
        self.label_causal = label_causal
        self.recommendations = recommendations
    
    def to_dict(self):
        """Convert to dictionary with JSON-safe types"""
        def make_json_safe(obj):
            if isinstance(obj, dict):
                return {k: make_json_safe(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [make_json_safe(item) for item in obj]
            elif isinstance(obj, (np.integer, np.int64, np.int32)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64, np.float32)):
                return float(obj)
            elif isinstance(obj, (np.bool_, bool)):
                return bool(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return obj
        
        data = {
            'alert_id': self.alert_id,
            'alert_data': self.alert_data,
            'xai_analysis': self.xai,
            'causal_analysis': self.causal,
            'label_causal': self.label_causal,
            'recommendations': self.recommendations
        }
        
        return make_json_safe(data)
    
    def to_json(self, filepath=None):
        """Export to JSON"""
        data = self.to_dict()
        if filepath:
            with open(filepath, 'w') as f:
                json.dump(data, f, indent=2)
            return filepath
        else:
            return json.dumps(data, indent=2)
    
    def to_text(self):
        """Generate natural language explanation"""
        lines = []
        lines.append("="*70)
        lines.append(f"HYBRID EXPLANATION - Alert #{self.alert_id or 'Unknown'}")
        lines.append("="*70)
        lines.append("")
        
        # Classification
        lines.append("🎯 CLASSIFICATION")
        lines.append(f"  Prediction: {self.xai['prediction']}")
        lines.append(f"  Confidence: {self.xai['confidence']:.1%}")
        lines.append(f"  Severity: {self.recommendations['severity']}")
        if self.recommendations.get('severity_reasoning'):
            lines.append(f"  Reasoning: {self.recommendations['severity_reasoning']}")
        lines.append(f"  Present Features: {self.xai['num_present_features']}/{len(FEATURE_NAMES)}")
        lines.append(f"  Missing Features: {self.xai['num_missing_features']}/{len(FEATURE_NAMES)}")
        lines.append("")
        
        # XAI Analysis
        lines.append("📊 XAI ANALYSIS (SHAP): What triggered this alert?")
        lines.append("  (Showing only features with values present in this alert)")
        lines.append("")
        for i, feat in enumerate(self.xai['top_features'], 1):
            missing_marker = " [MISSING]" if feat.get('is_missing', False) else ""
            lines.append(f"  {i}. {feat['feature']}{missing_marker}")
            lines.append(f"     Value: {feat['value']:.4f}")
            lines.append(f"     Importance: {feat['importance']:.4f}")
            lines.append("")
        
        # Causal Analysis
        lines.append("🔍 CAUSAL ANALYSIS: Why/How did this happen?")
        lines.append("  (Limited to SOC analyst-identified features)")
        lines.append("")
        
        has_causal_info = False
        for causal in self.causal:
            if not causal.get('in_graph', False):
                lines.append(f"  Feature: {causal['target']}")
                lines.append(f"  ⚠ {causal.get('reason', 'Not in causal graph')}")
                lines.append("")
                continue
            
            has_causal_info = True
            lines.append(f"  Feature: {causal['target']}")
            
            if causal.get('root_causes'):
                lines.append(f"  Root Causes: {', '.join(causal['root_causes'])}")
            
            if causal.get('causal_paths'):
                lines.append(f"  Causal Chains:")
                for path_info in causal['causal_paths'][:2]:
                    path_str = ' → '.join(path_info['path'])
                    lines.append(f"    • {path_str}")
            
            lines.append("")
        
        # Label causal
        if self.label_causal and self.label_causal.get('in_graph'):
            lines.append("  Direct Causes of Alert Classification:")
            for cause in self.label_causal.get('direct_causes', [])[:8]:
                value = cause['value']
                if isinstance(value, (int, float)) and value != -1.0:
                    lines.append(f"    • {cause['feature']} = {value:.4f}")
            lines.append("")
        
        # Recommendations
        lines.append("✅ RECOMMENDED ACTIONS")
        lines.append("")
        
        if self.recommendations['immediate_actions']:
            lines.append("  Immediate Actions:")
            for action in self.recommendations['immediate_actions']:
                lines.append(f"    • {action}")
            lines.append("")
        
        if self.recommendations['investigation_steps']:
            lines.append("  Investigation Steps:")
            for step in self.recommendations['investigation_steps']:
                lines.append(f"    • {step}")
            lines.append("")
        
        if self.recommendations['root_cause_mitigation']:
            lines.append("  Root Cause Mitigation:")
            for mitigation in self.recommendations['root_cause_mitigation']:
                lines.append(f"    • {mitigation}")
            lines.append("")
        
        lines.append("="*70)
        
        return '\n'.join(lines)
    
    def visualize(self, save_path=None):
        """Create visual explanation"""
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
        
        pred_emoji = "⚠️" if self.xai['prediction'] == 'Important' else "✓"
        fig.suptitle(f'{pred_emoji} Hybrid Explanation (SHAP) - Alert #{self.alert_id or "Unknown"}', 
                     fontsize=16, fontweight='bold')
        
        # 1. XAI Feature Importance
        ax1 = fig.add_subplot(gs[0, :])
        top_feats = self.xai['top_features']
        features = [f['feature'] for f in top_feats]
        importances = [f['importance'] for f in top_feats]
        
        colors = []
        for feat in top_feats:
            if feat.get('is_missing', False):
                colors.append('gray')
            elif feat['importance'] > 0:
                colors.append('red')
            else:
                colors.append('blue')
        
        ax1.barh(features, importances, color=colors, alpha=0.7)
        ax1.set_xlabel('SHAP Importance Score')
        ax1.set_title('XAI Analysis: Top Feature Importance (SHAP)', fontweight='bold')
        ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        ax1.grid(axis='x', alpha=0.3)
        
        # 2. Prediction info
        ax2 = fig.add_subplot(gs[1, 0])
        ax2.axis('off')
        
        severity_color = {
            'HIGH': '🔴', 'MEDIUM': '🟡', 'LOW': '🟢', 'MINIMAL': '⚪'
        }.get(self.recommendations['severity'], '⚪')
        
        pred_text = f"""
PREDICTION
{'='*30}
Class: {self.xai['prediction']}
Confidence: {self.xai['confidence']:.1%}
Severity: {severity_color} {self.recommendations['severity']}

Present Features: {self.xai['num_present_features']}/{len(FEATURE_NAMES)}
Missing Features: {self.xai['num_missing_features']}/{len(FEATURE_NAMES)}
"""
        ax2.text(0.1, 0.5, pred_text, fontsize=10, family='monospace',
                verticalalignment='center')
        
        # 3. Causal paths
        ax3 = fig.add_subplot(gs[1, 1])
        ax3.axis('off')
        
        causal_text = "CAUSAL CHAINS\n" + "="*30 + "\n"
        has_paths = False
        for causal in self.causal[:3]:
            if causal.get('in_graph') and causal.get('causal_paths'):
                has_paths = True
                for path_info in causal['causal_paths'][:1]:
                    path_str = ' → '.join(path_info['path'][:4])
                    if len(path_info['path']) > 4:
                        path_str += ' ...'
                    causal_text += f"• {path_str}\n"
        
        if not has_paths:
            causal_text += "\n⚠ Top features not in\ncausal graph\n"
        
        ax3.text(0.1, 0.5, causal_text, fontsize=10, family='monospace',
                verticalalignment='center')
        
        # 4. Recommendations
        ax4 = fig.add_subplot(gs[2, :])
        ax4.axis('off')
        
        rec_text = "RECOMMENDED ACTIONS\n" + "="*50 + "\n"
        if self.recommendations['immediate_actions']:
            rec_text += "\nImmediate:\n"
            for action in self.recommendations['immediate_actions'][:3]:
                shortened = action[:70] + "..." if len(action) > 70 else action
                rec_text += f"  • {shortened}\n"
        
        if self.recommendations['investigation_steps']:
            rec_text += "\nInvestigation:\n"
            for step in self.recommendations['investigation_steps'][:2]:
                shortened = step[:70] + "..." if len(step) > 70 else step
                rec_text += f"  • {shortened}\n"
        
        ax4.text(0.05, 0.5, rec_text, fontsize=9, family='monospace',
                verticalalignment='center')
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"✓ Saved visualization to: {save_path}")
        
        return fig

# ==================== DEMO ====================
print("\n" + "="*70)
print("GENERATING DEMO EXPLANATIONS WITH SHAP")
print("="*70)

explainer = HybridExplainer(
    model=model,
    causal_graph=causal_graph,
    feature_names=FEATURE_NAMES,
    scaler=scaler,
    shap_explainer=shap_explainer
)

if df is not None:
    print("\nGenerating explanations for sample alerts...")
    
    # Check label distribution
    if 'Label' in df.columns:
        label_counts = df['Label'].value_counts()
        print(f"\nDataset distribution:")
        for label, count in label_counts.items():
            pct = count / len(df) * 100
            print(f"  {label}: {count:,} ({pct:.1f}%)")
    
    # Select diverse samples
    important_samples = []
    irrelevant_samples = []
    
    if 'Label' in df.columns:
        sample_label = df['Label'].iloc[0]
        
        if isinstance(sample_label, (int, np.integer)):
            print("  Detected numeric labels (0=Normal, 1=Attack)")
            important_df = df[df['Label'] == 1]
            irrelevant_df = df[df['Label'] == 0]
        else:
            print("  Detected string labels")
            important_df = df[df['Label'] == 'Important']
            irrelevant_df = df[df['Label'] == 'Irrelevant']
        
        if len(important_df) > 0:
            important_samples = important_df.sample(n=min(3, len(important_df)), random_state=42).index.tolist()
        
        if len(irrelevant_df) > 0:
            irrelevant_samples = irrelevant_df.sample(n=min(2, len(irrelevant_df)), random_state=42).index.tolist()
        
        sample_indices = important_samples + irrelevant_samples
        print(f"\nSelected {len(important_samples)} Important + {len(irrelevant_samples)} Normal alerts")
    else:
        sample_indices = [0, 100, 500]
    
    for idx in sample_indices:
        if idx >= len(df):
            continue
        
        print(f"\n{'='*70}")
        true_label = df.iloc[idx].get('Label', 'Unknown')
        if isinstance(true_label, (int, np.integer)):
            true_label_str = f"{true_label}"
        else:
            true_label_str = str(true_label)
        print(f"EXAMPLE ALERT #{idx} (True Label: {true_label_str})")
        print(f"{'='*70}")
        
        # Get alert data
        alert_row = df.iloc[idx]
        alert_features = alert_row[FEATURE_NAMES].values
        
        # Generate explanation
        explanation = explainer.explain(alert_features, alert_id=idx)
        
        # Print text explanation
        print(explanation.to_text())
        
        # Save JSON
        json_path = f'hybrid_explanation_shap_{idx}.json'
        explanation.to_json(json_path)
        print(f"\n✓ Saved JSON to: {json_path}")
        
        # Save visualization
        viz_path = f'hybrid_explanation_shap_{idx}.png'
        explanation.visualize(save_path=viz_path)
        plt.close()

print("\n" + "="*70)
print("STEP 4 COMPLETE - USING SHAP")
print("="*70)
print("\nGenerated files:")
print("  - hybrid_explanation_shap_*.json (structured data)")
print("  - hybrid_explanation_shap_*.png (visualizations)")
print("\nKey differences from DeepLIFT version:")
print("  ✓ Uses SHAP KernelExplainer for model-agnostic explanations")
print("  ✓ Background data: 100 samples from training set")
print("  ✓ Same output format for Step 5 evaluation")
print("\nNext: Step 5 - Evaluation (quantitative metrics + comparison)")